**XRPL Basic Transaction Test**
============================
This script demonstrates:
1. Connecting to XRPL Testnet
2. Creating and funding two wallets
3. Checking balances
4. Sending XRP between wallets
5. Verifying the transaction

In [22]:
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.asyncio.wallet import generate_faucet_wallet
from xrpl.models.transactions import Payment
from xrpl.models.requests import AccountInfo
from xrpl.asyncio.transaction import submit_and_wait
from xrpl.utils import drops_to_xrp, xrp_to_drops

In [23]:
# =============================================================================
# STEP 1: Connect to Testnet
# =============================================================================

TESTNET_URL = "https://s.altnet.rippletest.net:51234"

print("=" * 60)
print("XRPL BASIC TRANSACTION TEST")
print("=" * 60)

print("\n[1] Connecting to XRPL Testnet...")
client = AsyncJsonRpcClient(TESTNET_URL)
print(f"    ✓ Connected to {TESTNET_URL}")

XRPL BASIC TRANSACTION TEST

[1] Connecting to XRPL Testnet...
    ✓ Connected to https://s.altnet.rippletest.net:51234


In [24]:

# =============================================================================
# STEP 2: Create and Fund Two Wallets
# =============================================================================

print("\n[2] Creating wallets (this may take a moment)...")

print("\n    Creating Wallet A (sender)...")
wallet_a = await generate_faucet_wallet(client, debug=True)

print("\n    Creating Wallet B (receiver)...")
wallet_b = await generate_faucet_wallet(client, debug=True)

print(f"\n    ✓ Wallet A: {wallet_a.address}")
print(f"    ✓ Wallet B: {wallet_b.address}")



[2] Creating wallets (this may take a moment)...

    Creating Wallet A (sender)...
Attempting to fund address rDRJyeVcznTfwmAPEogC1tXTY9vJGr5vpd
Faucet fund successful.

    Creating Wallet B (receiver)...
Attempting to fund address rZoS42awTMyGekdPvman5gKTY81QKpcUe
Faucet fund successful.

    ✓ Wallet A: rDRJyeVcznTfwmAPEogC1tXTY9vJGr5vpd
    ✓ Wallet B: rZoS42awTMyGekdPvman5gKTY81QKpcUe


In [ ]:
# =============================================================================
# STEP 3: Check Initial Balances
# =============================================================================

async def get_balance(client, address):
    """Get XRP balance for an address (returns in XRP, not drops)."""
    request = AccountInfo(account=address, ledger_index="validated")
    response = await client.request(request)
    balance_drops = response.result["account_data"]["Balance"]
    return drops_to_xrp(balance_drops)

print("\n[3] Checking balances...")

balance_a_before = await get_balance(client, wallet_a.address)
balance_b_before = await get_balance(client, wallet_b.address)

print(f"    Wallet A: {balance_a_before} XRP")
print(f"    Wallet B: {balance_b_before} XRP")



[3] Checking initial balances...
    Wallet A: 100.000000 XRP
    Wallet B: 100.000000 XRP


In [26]:
# =============================================================================
# STEP 4: Send XRP from Wallet A to Wallet B
# =============================================================================

SEND_AMOUNT_XRP = 50  # Amount to send

print(f"\n[4] Sending {SEND_AMOUNT_XRP} XRP from Wallet A to Wallet B...")

# Build the payment transaction
payment_tx = Payment(
    account=wallet_a.address,           # Sender
    destination=wallet_b.address,       # Receiver
    amount=xrp_to_drops(SEND_AMOUNT_XRP),  # Amount in drops (1 XRP = 1,000,000 drops)
)

print(f"    Transaction built:")
print(f"      From:   {wallet_a.address}")
print(f"      To:     {wallet_b.address}")
print(f"      Amount: {SEND_AMOUNT_XRP} XRP ({xrp_to_drops(SEND_AMOUNT_XRP)} drops)")

# Submit and wait for validation
print("\n    Submitting transaction...")
response = await submit_and_wait(payment_tx, client, wallet_a)

# Check the result
tx_result = response.result["meta"]["TransactionResult"]
print(f"\n    Transaction result: {tx_result}")

if tx_result == "tesSUCCESS":
    print("    ✓ Transaction successful!")
    
    # Get transaction details
    tx_hash = response.result["hash"]
    fee_drops = response.result["tx_json"]["Fee"]
    
    print(f"\n    Transaction details:")
    print(f"      Hash: {tx_hash}")
    print(f"      Fee:  {drops_to_xrp(fee_drops)} XRP ({fee_drops} drops)")
    print(f"\n    View on explorer:")
    print(f"      https://testnet.xrpl.org/transactions/{tx_hash}")
else:
    print(f"    ✗ Transaction failed: {tx_result}")



[4] Sending 50 XRP from Wallet A to Wallet B...
    Transaction built:
      From:   rDRJyeVcznTfwmAPEogC1tXTY9vJGr5vpd
      To:     rZoS42awTMyGekdPvman5gKTY81QKpcUe
      Amount: 50 XRP (50000000 drops)

    Submitting transaction...

    Transaction result: tesSUCCESS
    ✓ Transaction successful!

    Transaction details:
      Hash: CEC402EC8FFE26667DC271110728DA09EB311039ECE24E5005A577D2E8104D9C
      Fee:  0.000010 XRP (10 drops)

    View on explorer:
      https://testnet.xrpl.org/transactions/CEC402EC8FFE26667DC271110728DA09EB311039ECE24E5005A577D2E8104D9C


In [27]:
import json

print(json.dumps(response.result, indent=2))

{
  "close_time_iso": "2026-01-15T00:39:10Z",
  "ctid": "C0D66EBE00000001",
  "hash": "CEC402EC8FFE26667DC271110728DA09EB311039ECE24E5005A577D2E8104D9C",
  "ledger_hash": "748113879FBAEA8215D27FDF44F2AE5D8E7B71B83B4DC04AADAEFBA231EA47DC",
  "ledger_index": 14053054,
  "meta": {
    "AffectedNodes": [
      {
        "ModifiedNode": {
          "FinalFields": {
            "Account": "rZoS42awTMyGekdPvman5gKTY81QKpcUe",
            "Balance": "150000000",
            "Flags": 0,
            "OwnerCount": 0,
            "Sequence": 14053052
          },
          "LedgerEntryType": "AccountRoot",
          "LedgerIndex": "1C508CBB4628CFF75D007CA94692DE8C607C634CC0D5DB63B1665C30AE422A09",
          "PreviousFields": {
            "Balance": "100000000"
          },
          "PreviousTxnID": "8B533F04A7275956FFDFCBB7678A5D6998744EFA39793C6169DB6640E70B72AE",
          "PreviousTxnLgrSeq": 14053052
        }
      },
      {
        "ModifiedNode": {
          "FinalFields": {
            

In [28]:
# =============================================================================
# STEP 5: Verify Final Balances
# =============================================================================

print("\n[5] Checking final balances...")

balance_a_after = await get_balance(client, wallet_a.address)
balance_b_after = await get_balance(client, wallet_b.address)

print(f"    Wallet A: {balance_a_after} XRP (was {balance_a_before} XRP)")
print(f"    Wallet B: {balance_b_after} XRP (was {balance_b_before} XRP)")

# Calculate actual changes
change_a = float(balance_a_after) - float(balance_a_before)
change_b = float(balance_b_after) - float(balance_b_before)

print(f"\n    Balance changes:")
print(f"      Wallet A: {change_a:+.6f} XRP (sent {SEND_AMOUNT_XRP} + fee)")
print(f"      Wallet B: {change_b:+.6f} XRP (received {SEND_AMOUNT_XRP})")



[5] Checking final balances...
    Wallet A: 49.999990 XRP (was 100.000000 XRP)
    Wallet B: 150.000000 XRP (was 100.000000 XRP)

    Balance changes:
      Wallet A: -50.000010 XRP (sent 50 + fee)
      Wallet B: +50.000000 XRP (received 50)


In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"""
Wallets Created:
  Wallet A: {wallet_a.address}
  Wallet B: {wallet_b.address}

Transaction:
  Sent {SEND_AMOUNT_XRP} XRP from A to B
  Result: {tx_result}

View accounts on testnet explorer:
  Wallet A: https://testnet.xrpl.org/accounts/{wallet_a.address}
  Wallet B: https://testnet.xrpl.org/accounts/{wallet_b.address}

""")

print("✓ Test complete!")


SUMMARY

Wallets Created:
  Wallet A: rDRJyeVcznTfwmAPEogC1tXTY9vJGr5vpd
  Wallet B: rZoS42awTMyGekdPvman5gKTY81QKpcUe

Transaction:
  Sent 50 XRP from A to B
  Result: tesSUCCESS

View accounts on testnet explorer:
  Wallet A: https://testnet.xrpl.org/accounts/rDRJyeVcznTfwmAPEogC1tXTY9vJGr5vpd
  Wallet B: https://testnet.xrpl.org/accounts/rZoS42awTMyGekdPvman5gKTY81QKpcUe

Note: These are testnet wallets with test XRP (no real value).
      Testnet may reset periodically, so don't rely on persistence.

✓ Test complete!
